In [ ]:
%reload_ext autoreload
%autoreload 2

import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from networkx.algorithms.components import strongly_connected_component_subgraphs
from networkx.algorithms.components import connected_component_subgraphs
import timeit
import os


import pickle
import time

workdir = "/home/michele/aalto/dm/DMproject/dmproject/approximate"
if(os.curdir != workdir):
    os.chdir("/home/michele/aalto/dm/DMproject/dmproject/approximate")

import sys
sys.path.insert(0, '/home/michele/aalto/dm/DMproject/dmproject/utils')

import utils

In [ ]:
graph_files = []
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'wiki_vote', 'Wiki-Vote.txt'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'epinions', 'soc-Epinions1.txt'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'gplus', 'gplus_combined.txt'))
graph_files.append(os.path.join(os.path.pardir, 'dataset', 'soc_pokec', 'soc-pokec-relationships.txt'))

graph_names = ["wikivote", "epinions", "gplus", "pokec"]

In [ ]:
graphname = graph_names[0]
file = graph_files[0]
#fh = open(file, "r")
# import of directed
dG = nx.read_adjlist(file, create_using=nx.DiGraph())
# conversion to undirected graph
G = dG.to_undirected()

In [ ]:
np.random.seed(132435)
p_samples = [0.05]#, 0.4, 0.7]
n_repetitions = [5]#, 100, 500, 1000]

dim = len(G.nodes)

matrix_cc = utils.reservoir_sampling_nodes(G, dim, n_repetitions, p_samples, False)
matrix_scc = utils.reservoir_sampling_nodes(dG, dim, n_repetitions, p_samples, True)
    

In [ ]:
def all_pairs_shortest_path_bfs(G, selected_nodes):
    #if isinstance(G, nx.DiGraph):   
    #else:
    shortest_path_mat = []
    for node in selected_nodes:
        shortest_path_len = list(nx.single_source_shortest_path(G, node))
        
        print(shortest_path_len)
        sys.exit(0)
        
        shortest_path_mat.append(shortest_path_len)
    
        

stats_cc_p = []
stats_scc_p = []
for i, p in enumerate(p_samples):
    stats_cc_aggr = []
    stats_scc_aggr = []
    for j, n in enumerate(n_repetitions):
        stats_cc_n = []
        stats_scc_n = []
        for h in range(n):
            selected_nodes_cc = matrix_cc[i][j][h]
            selected_nodes_scc =matrix_scc[i][j][h]
            stats_cc = all_pairs_shortest_path_bfs(G, selected_nodes_cc)
            stats_scc = all_pairs_shortest_path_bfs(dG, selected_nodes_scc)
            stats_cc_n.append(stats_cc)
            stats_scc_n.append(stats_scc)
            print(graphname + "_" + str(p) + "_" + str(n) + "_" + str(h))
        stats_cc_n = np.array(stats_cc_n)
        stats_scc_n = np.array(stats_scc_n)
        
        aggr_stats_cc = utils.aggregate_stats(stats_cc_n)
        aggr_stats_scc = utils.aggregate_stats(stats_scc_n)
        
        stats_cc_aggr.append(aggr_stats_cc)
        stats_scc_aggr.append(aggr_stats_scc)
    stats_cc_p.append(stats_cc_aggr)
    stats_scc_p.append(stats_scc_aggr)